In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers
!pip install pytorch_lightning
!pip install multilingual-clip
!pip install langchain unstructured[all-docs] pydantic lxml

In [ ]:
%cd /content
!git clone -b v1.0 https://github.com/camenduru/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2
!pip install -e .

In [ ]:
import warnings
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [ ]:
import torch
import clip
# !pip install git+https://github.com/openai/CLIP.git
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# clip_model, compose = clip.load('ViT-L/14', device = device)
# text_model = text_model.cpu()
# def process(idx_val,arr):
#   if idx_val=='0':
#     arr.append(0)
#   else:
#     arr.append(1)

In [ ]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import torch
import math
from torch import nn
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score,precision_score
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from PIL import Image
from torchvision import transforms
from PIL import ImageFile
import fireworks.client
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
data = pd.read_csv('/content/Multitasking_Persuasive_Memes.csv')

In [ ]:
data.head(10)

In [ ]:
from multilingual_clip import pt_multilingual_clip
import transformers

In [ ]:
#just to check how multilingual clip works
texts = [
    'Three blind horses listening to Mozart.',
    'Älgen är skogens konung!',
    'Wie leben Eisbären in der Antarktis?',
    'Вы знали, что все белые медведи левши?'
]
model_name = 'M-CLIP/XLM-Roberta-Large-Vit-L-14'
# Load Model & Tokenizer
model = pt_multilingual_clip.MultilingualCLIP.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
embeddings = model.forward(texts, tokenizer)
print(embeddings.shape)

In [ ]:
sample = data['text'][10]
sample

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = preprocess(Image.open('/content/drive/MyDrive/Gitanjali Mam/persuasive_meme/aloknath0.png')).unsqueeze(0).to(device)
text = clip.tokenize([sample]).to(device)

In [ ]:
text

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']='2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# clip_model, compose = clip.load('RN50x4', device = device)
clip_model, compose = clip.load("ViT-B/32", device = device)
text_inputs = (clip.tokenize(data.text.values[321],truncate=True)).to(device)
print(text_inputs)

In [ ]:
len(data)

In [ ]:
data.Name.values[1:10]

In [ ]:
data.head()

In [ ]:
import fireworks.client
import os
fireworks.client.api_key = os.getenv("zM8J4Kh5Ob3QGw4I3aN6xiX3GUee8Ola7wnMHat9cutf7XFx")
fireworks.client.api_key = "zM8J4Kh5Ob3QGw4I3aN6xiX3GUee8Ola7wnMHat9cutf7XFx"

def get_completion(prompt, model=None, max_tokens=50):
    fw_model_dir = "accounts/fireworks/models/"
    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model
    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        n=1,
        max_tokens=200,
        temperature=0.1,
        top_p=0.9
    )
    return completion.choices[0].text

def get_prompt_text(text_re):
  prompt1 = f"Retrieve relevant information about the meme with the text transcription: {text_re}  Explain the sentiment with context and any additional insights associated with this meme"
  return prompt1

prompt1 = "Retrieve relevant information about the meme with the text transcription: Explain the sentiment with context and any additional insights associated with this meme"
mistral_llm = "mistral-7b-instruct-4k"
get_completion(prompt1, model=mistral_llm)

#get_completion(prompt1+ data['Text_Transcription'][10], model=mistral_llm)

In [ ]:
data.head()

In [ ]:
def get_data(data):
  #data = pd.read_csv(dataset_path)
  text = list(data['text'])
  img_path = list(data['Name'])
  name = list(data['Name'])
  persuasive_inten = list(data['persuasive_inten'])
  #persuasive_inten = list(map(lambda x: x - 1 , persuasive_inten))
  label = list(data['Persuasive'])
  per_intem=list(data['persuasive_inten'])
  t3_1 =list(data['Irony'])
  t3_2 = list(data['personification'])
  t3_3 = list(data['Alliteration'])
  t3_4 = list(data['Analogies'])
  t3_5 = list(data['Invective'])
  t3_6 = list(data['Metaphor'])
  t3_7 = list(data['puns_and_wordplays'])
  t3_8 = list(data['Satire'])
  t3_9 = list(data['Hyperboles'])
  t1 = list(data['None'])
  t2 = list(data['Negatively persuasive'])
  t3 = list(data['Slightly Negatively persuasive'])
  t4 = list(data['Neutral'])
  t5 = list(data['Positively persuasive'])
  t6 = list(data['Slightly Positively persuasive'])

  text_features,image_features,rag_features,Name,l,ir,per,alli,ana,inv,meta,puaps,sat,hyp,persi,a_1,a_2,a_3,a_4,a_5,a_6 = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

  for txt,img,L,n,a,b,c,d,e,f,g,h,i,v,j,k,q,m,n,o in tqdm(zip(text,img_path,label,name,t3_1,t3_2,t3_3,t3_4,t3_5,t3_6,t3_7,t3_8,t3_9,persuasive_inten,t1,t2,t3,t4,t5,t6)):
    try:
      img = Image.open('/content/drive/MyDrive/Gitanjali Mam/persuasive_meme/'+img)
    except Exception as e:
      print(e)
      continue
    txt_rag = get_completion(get_prompt_text(txt)+txt, model=mistral_llm)
    txt2 = txt_rag
    img = torch.stack([compose(img).to(device)])
    l.append(L)
    Name.append(n)
    ir.append(a)
    persi.append(v)
    per.append(b)
    alli.append(c)
    ana.append(d)
    inv.append(e)
    meta.append(f)
    puaps.append(g)
    sat.append(h)
    hyp.append(i)
    a_1.append(j)
    a_2.append(k)
    a_3.append(q)
    a_4.append(m)
    a_5.append(n)
    a_6.append(o)

    with torch.no_grad():
      temp_rag=model.forward(txt2, tokenizer).detach().cpu().numpy()
      rag_features.append(temp_rag)
      temp_txt=model.forward(txt, tokenizer).detach().cpu().numpy()
      text_features.append(temp_txt)
      temp_img = clip_model.encode_image(img).detach().cpu().numpy()
      image_features.append(temp_img)
      del temp_txt
      del temp_img
      torch.cuda.empty_cache()
    del img
    torch.cuda.empty_cache()
  return text_features,rag_features,image_features,l,Name,ir,per,alli,ana,inv,meta,puaps,sat,hyp,persi,a_1,a_2,a_3,a_4,a_5,a_6


#Pre-Processing:
#Converts the opened image (img) to a PyTorch tensor and stacks it into a batch
#Uses CLIP to encode text into text_features & image to image_features
#CLIP Uses Zero-Shot Learning_

In [ ]:
#trial
text_features,rag_features,image_features,l,Name,ir,per,alli,ana,inv,meta,puaps,sat,hyp,persi,a_1,a_2,a_3,a_4,a_5,a_6 = get_data(data.head(5))

In [ ]:
outliers = []
for names in tqdm(list(data['Name'])):
  #change the path according to your drive
  if not os.path.exists('/content/drive/MyDrive/Gitanjali Mam/persuasive_meme/'+names):
    outliers.append(names)

# data = data[~data['Name'].isin(outliers)]

In [ ]:
class HatefulDataset(Dataset):

  def __init__(self,data):

    self.t_f,self.r_f,self.i_f,self.label,self.name,self.persuasive_inten,\
     self.t3_1, self.t3_2, self.t3_3, self.t3_4, self.t3_5,\
     self.t3_6, self.t3_7, self.t3_8, self.t3_9,self.t1, self.t2, self.t3,self.t4, self.t5, self.t6= get_data(data)
    self.t_f = np.squeeze(np.asarray(self.t_f),axis=1)
    self.r_f = np.squeeze(np.asarray(self.r_f),axis=1)
    self.i_f = np.squeeze(np.asarray(self.i_f),axis=1)

  def __len__(self):
    return len(self.a)

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    #print(idx)
    T = self.t_f[idx,:]
    R = self.r_f[idx,:]
    I = self.i_f[idx,:]
    name=self.name[idx]
    label = self.label[idx]
    persuasive_inten = self.persuasive_inten[idx]

    t3_1 = self.t3_1[idx]
    t3_2 = self.t3_2[idx]
    t3_3 = self.t3_1[idx]
    t3_4 = self.t3_4[idx]
    t3_5 = self.t3_5[idx]
    t3_6 = self.t3_6[idx]
    t3_7 = self.t3_7[idx]
    t3_8 = self.t3_8[idx]
    t3_9 = self.t3_9[idx]
    t1 = self.t1[idx]
    t2 = self.t2[idx]
    t3 = self.t3[idx]
    t4 = self.t4[idx]
    t5 = self.t5[idx]
    t6 = self.t6[idx]

    #name = self.name[idx]

    sample = {'label':label,'processed_txt':T,'processed_rag':R,'processed_img':I,'name':name,'persuasive_inten':persuasive_inten,'irony':t3_1,'personification':t3_2,'Alliteration': t3_3,'Analogies':t3_4,
              'Invective':t3_5,'Metaphor':t3_5,'puns_and_wordplays': t3_7,'Satire':t3_8,'Hyperboles':t3_9,'t1':t1, 't2':t2, 't3':t3, 't4':t4, 't5':t5, 't6':t6
              }
    return sample

In [ ]:
sample_dataset = HatefulDataset(data.head(500))

In [ ]:
torch.save(sample_dataset,'/content/persuasive_train50.pt')
sample_dataset_new= torch.load("/content/new4000.pt")
test= torch.load("/content/new4000.pt")
len(sample_dataset_new)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MFB(nn.Module):
    def __init__(self,img_feat_size, ques_feat_size, is_first, MFB_K, MFB_O, DROPOUT_R):
        super(MFB, self).__init__()
        #self.__C = __C
        self.MFB_K = MFB_K
        self.MFB_O = MFB_O
        self.DROPOUT_R = DROPOUT_R
        self.is_first = is_first
        self.proj_i = nn.Linear(img_feat_size, MFB_K * MFB_O)
        self.proj_q = nn.Linear(ques_feat_size, MFB_K * MFB_O)
        self.dropout = nn.Dropout(DROPOUT_R)
        self.pool = nn.AvgPool1d(MFB_K, stride = MFB_K)

    def forward(self, img_feat, ques_feat, exp_in=1):
        batch_size = img_feat.shape[0]
        img_feat = self.proj_i(img_feat)                # (N, C, K*O)
        ques_feat = self.proj_q(ques_feat)              # (N, 1, K*O)

        exp_out = img_feat * ques_feat             # (N, C, K*O)
        exp_out = self.dropout(exp_out) if self.is_first else self.dropout(exp_out * exp_in)     # (N, C, K*O)
        z = self.pool(exp_out) * self.MFB_K         # (N, C, O)
        z = torch.sqrt(F.relu(z)) - torch.sqrt(F.relu(-z))
        z = F.normalize(z.view(batch_size, -1))         # (N, C*O)
        z = z.view(batch_size, -1, self.MFB_O)      # (N, C, O)
        return z


#MFB -> Multimodal Factorized Bilinear Pooling
#used to model complex interactions between features like image and text
#MFB_K -> Number Of factors, MFB_O -> Output size,
#Init initializes linear projection layers for image and question features , dropout layer and average pooling layer

#Forward:

#exp_in = input expansion factor (default - 1)
#Linear projection of image and question features to factorized bilinear form
#Element-wise multiplication of image and question features
#APply Dropout
#Average pooling along the factorized dimension (MFB_K) to reduce the size of the output tensor
#Element-wise operations to compute the final output (z) using square root and normalization using Relu.
#The final output represents the fused representation of image and question features.


In [ ]:
data = data[~data['Name'].isin(outliers)]
len(sample_dataset_new)

In [ ]:
torch.manual_seed(123)
t_p,v_p = torch.utils.data.random_split(sample_dataset_new,[3500,500])

# torch.manual_seed(123)
t_p,te_p = torch.utils.data.random_split(t_p,[3000,500])

In [ ]:
t_p[1]["processed_img"].shape
t_p[1]['processed_txt'].shape
t_p[1]['processed_rag'].shape

In [ ]:
class Classifier(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.MFB = MFB(512,768,True,192,64,0.1)
    self.fin_y_shape = torch.nn.Linear(768,512)
    self.fin_old = torch.nn.Linear(2048,2)
    self.fin_e = nn.Linear(16 * 768, 64)  # Adjusted input size to match the reshaped output from MFB
    self.fin_inten = torch.nn.Linear(2048,6)
    self.fin_e1 = torch.nn.Linear(64,2)
    self.fin_e2 = torch.nn.Linear(64,2)
    self.fin_e3 = torch.nn.Linear(64,2)
    self.fin_e4 = torch.nn.Linear(64,2)
    self.fin_e5 = torch.nn.Linear(64,2)
    self.fin_e6 = torch.nn.Linear(64,2)
    self.fin_e7 = torch.nn.Linear(64,2)
    self.fin_e8 = torch.nn.Linear(64,2)
    self.fin_e9 = torch.nn.Linear(64,2)
    self.validation_step_outputs = []
    self.test_step_outputs = []

  def forward(self,x,y,rag):
      x_,y_,rag_ = x,y,rag
      z= torch.cat((x, y, rag), dim=1)
      z_new = torch.squeeze(z, dim=1)
      c_inten = self.fin_inten(z_new)
      c_e1 = self.fin_e1(z_new)
      c_e2 = self.fin_e2(z_new)
      c_e3 = self.fin_e3(z_new)
      c_e4 = self.fin_e4(z_new)
      c_e5 = self.fin_e5(z_new)
      c_e6 = self.fin_e6(z_new)
      c_e7 = self.fin_e7(z_new)
      c_e8 = self.fin_e8(z_new)
      c_e9 = self.fin_e9(z_new)
      c = self.fin_old(z_new)

      # probability distribution over labels
      c = torch.log_softmax(c, dim=1)
      c_inten = torch.log_softmax(c_inten, dim=1)
      c_e1 = torch.log_softmax(c_e1, dim=1)
      c_e2 = torch.log_softmax(c_e2, dim=1)
      c_e3 = torch.log_softmax(c_e3, dim=1)
      c_e4 = torch.log_softmax(c_e4, dim=1)
      c_e5 = torch.log_softmax(c_e5, dim=1)
      c_e6 = torch.log_softmax(c_e6, dim=1)
      c_e7 = torch.log_softmax(c_e7, dim=1)
      c_e8 = torch.log_softmax(c_e8, dim=1)
      c_e9 = torch.log_softmax(c_e9, dim=1)

      return z,c,c_a,c_v,c_e1,c_e2,c_e3,c_e4,c_e5,c_e6,c_e7,c_e8,c_e9,c_inten

  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)


  def training_step(self, train_batch, batch_idx):
      lab,txt,rag,img,name,intensity,e1,e2,e3,e4,e5,e6,e7,e8,e9,t1,t2,t3,t4,t5,t6,t7 = train_batch
      lab = train_batch[lab]
      txt = train_batch[txt]
      rag = train_batch[rag]
      img = train_batch[img]
      name= train_batch[name]
      intensity = train_batch[intensity]
      e1 = train_batch[e1]
      e2 = train_batch[e2]
      e3 = train_batch[e3]
      e4 = train_batch[e4]
      e5 = train_batch[e5]
      e6 = train_batch[e6]
      e7 = train_batch[e7]
      e8 = train_batch[e8]
      e9 = train_batch[e9]
      t1,t2,t3,t4,t5,t6,t7 = torch.unsqueeze(batch[t1],1),torch.unsqueeze(batch[t2],1),\
      torch.unsqueeze(batch[t3],1),torch.unsqueeze(batch[t4],1),torch.unsqueeze(batch[t5],1),\
      torch.unsqueeze(batch[t6],1),torch.unsqueeze(batch[t7],1)
      gt_target = torch.cat((t1,t2,t3,t4,t5,t6),1)
      gt_emotion = torch.cat((torch.unsqueeze(e1,1),torch.unsqueeze(e2,1),torch.unsqueeze(e3,1),torch.unsqueeze(e4,1),torch.unsqueeze(e5,1),torch.unsqueeze(e6,1),\
                              torch.unsqueeze(e7,1),torch.unsqueeze(e8,1),torch.unsqueeze(e9,1)),1)

      z,logit_offen, a,b,c,d,e,f,g,h,i,j,k,l,m,logitinten_target,logit_sarcasm,logit_emotion= self.forward(txt,img,rag)

      loss1 = self.cross_entropy_loss(logit_offen, lab)
      #loss2 = self.cross_entropy_loss(logit_arou, arou)
      #loss3 = self.cross_entropy_loss(logit_val, val)
      loss4 = self.cross_entropy_loss(a, e1)
      loss5 = self.cross_entropy_loss(b, e2)
      loss6 = self.cross_entropy_loss(c, e3)
      loss7 = self.cross_entropy_loss(d, e4)
      loss8 = self.cross_entropy_loss(e, e5)
      loss9 = self.cross_entropy_loss(f, e6)

      loss17 = self.cross_entropy_loss(inten, intensity)

      loss18 = F.binary_cross_entropy_with_logits(logit_target.float(), gt_target.float())
      loss_emo_mult = F.binary_cross_entropy_with_logits(logit_emotion.float(), gt_emotion.float())


      #loss = loss1+loss_emo_mult+loss17
      loss=loss1+loss_emo_mult
      self.log('train_loss', loss)

      return loss


  def validation_step(self, val_batch, batch_idx):
      lab,txt,img,e1,e2,e3,e4,e5,e6,e7,e8,e9,intensity,t1,t2,t3,t4,t5,t6 = val_batch

      #print(val_batch)
      lab = val_batch[lab]
      txt = val_batch[txt]
      img = val_batch[img]

      e1 = val_batch[e1]
      e2 = val_batch[e2]
      e3 = val_batch[e3]
      e4 = val_batch[e4]
      e5 = val_batch[e5]
      e6 = val_batch[e6]
      e7 = val_batch[e7]
      e8 = val_batch[e8]
      e9 = val_batch[e9]


      intensity = val_batch[intensity]
      t1,t2,t3,t4,t5,t6,t7 = torch.unsqueeze(val_batch[t1],1),torch.unsqueeze(val_batch[t2],1),\
      torch.unsqueeze(val_batch[t3],1),torch.unsqueeze(val_batch[t4],1),torch.unsqueeze(val_batch[t5],1),\
      torch.unsqueeze(val_batch[t6],1)
      #print(t1.size())
      gt_target = torch.cat((t1,t2,t3,t4,t5,t6,t7),1) #ground truth target
      gt_emotion = torch.cat((torch.unsqueeze(e1,1),torch.unsqueeze(e2,1),torch.unsqueeze(e3,1),torch.unsqueeze(e4,1),torch.unsqueeze(e5,1),torch.unsqueeze(e6,1),\
                              torch.unsqueeze(e7,1),torch.unsqueeze(e8,1),torch.unsqueeze(e9,1)),1)

      logits, a,b,c,d,e,f,g,h,i,j,k,l,m,inten,logit_target,logit_emotion = self.forward(txt,img,rag)





      tmp = np.argmax(logits.detach().cpu().numpy(),axis=-1)
      loss = self.cross_entropy_loss(logits, lab)
      lab = lab.detach().cpu().numpy()
      self.log('val_acc', f1_score(lab,tmp,average='macro'))
      #self.log('val_roc_auc',roc_auc_score(lab,tmp))
      self.log('val_loss', loss)
      tqdm_dict = {'val_acc': accuracy_score(lab,tmp)}
      #print('Val acc {}'.format(accuracy_score(lab,tmp)))
      return {
                'progress_bar': tqdm_dict,
              'val_loss_target': F.binary_cross_entropy_with_logits(logit_target.float(), gt_target.float()),
              'val_loss_emotion_multilabel': F.binary_cross_entropy_with_logits(logit_emotion.float(), gt_emotion.float()),
              'val_acc e1': accuracy_score(e1.detach().cpu().numpy(),np.argmax(a.detach().cpu().numpy(),axis=-1)),
      'val_acc e2': accuracy_score(e2.detach().cpu().numpy(),np.argmax(b.detach().cpu().numpy(),axis=-1)),
      'val_acc e3': accuracy_score(e3.detach().cpu().numpy(),np.argmax(c.detach().cpu().numpy(),axis=-1)),
      'val_acc e4': accuracy_score(e4.detach().cpu().numpy(),np.argmax(d.detach().cpu().numpy(),axis=-1)),
      'val_acc e5': accuracy_score(e5.detach().cpu().numpy(),np.argmax(e.detach().cpu().numpy(),axis=-1)),
      'val_acc e6': accuracy_score(e6.detach().cpu().numpy(),np.argmax(f.detach().cpu().numpy(),axis=-1)),
      'val_acc e7': accuracy_score(e7.detach().cpu().numpy(),np.argmax(g.detach().cpu().numpy(),axis=-1)),
      'val_acc e8': accuracy_score(e8.detach().cpu().numpy(),np.argmax(h.detach().cpu().numpy(),axis=-1)),
      'val_acc e9': accuracy_score(e9.detach().cpu().numpy(),np.argmax(i.detach().cpu().numpy(),axis=-1)),
      'val_acc intensity': f1_score(intensity.detach().cpu().numpy(),np.argmax(inten.detach().cpu().numpy(),axis=-1),average='macro'),
      'f1 sarcasm': f1_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1),average='macro')
      }

  def validation_epoch_end(self, validation_step_outputs):
    outs = []
    outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14,outs16,outs17 = \
    [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    outs15 = []
    outs18 = []
    for out in validation_step_outputs:
      outs.append(out['progress_bar']['val_acc'])
      outs1.append(out['val_acc e1'])
      outs2.append(out['val_acc e2'])
      outs3.append(out['val_acc e3'])
      outs4.append(out['val_acc e4'])
      outs5.append(out['val_acc e5'])
      outs6.append(out['val_acc e6'])
      outs7.append(out['val_acc e7'])
      outs8.append(out['val_acc e8'])
      outs9.append(out['val_acc e9'])

      outs14.append(out['val_acc intensity'])
      outs15.append(out['val_loss_target'])
      outs16.append(out['val_loss_emotion_multilabel'])
      outs17.append(out['val_acc sarcasm'])
      outs18.append(out['f1 sarcasm'])
    self.log('val_acc_all_offn', sum(outs)/len(outs))
    self.log('val_loss_target', sum(outs15)/len(outs15))
    self.log('val_acc_all e1', sum(outs1)/len(outs1))
    self.log('val_acc_all e2', sum(outs2)/len(outs2))
    self.log('val_acc_all e3', sum(outs3)/len(outs3))
    self.log('val_acc_all e4', sum(outs4)/len(outs4))
    self.log('val_acc_all e5', sum(outs5)/len(outs5))
    self.log('val_acc_all e6', sum(outs6)/len(outs6))
    self.log('val_acc_all e7', sum(outs7)/len(outs7))
    self.log('val_acc_all e8', sum(outs8)/len(outs8))
    self.log('val_acc_all e9', sum(outs9)/len(outs9))

    self.log('val_acc_all inten', sum(outs14)/len(outs14))
    self.log('val_loss_all emo', sum(outs16)/len(outs16))


    print(f'***offensive f1 at epoch end {sum(outs)/len(outs)}****')
    #print(f'***val acc inten at epoch end {sum(outs14)/len(outs14)}****')
    print(f'***val loss emotion at epoch end {sum(outs16)/len(outs16)}****')
    #print(f'***val acc sarcasm at epoch end {sum(outs17)/len(outs17)}****')
    #print(f'***val f1 sarcasm at epoch end {sum(outs18)/len(outs18)}****')

  def test_step(self, batch, batch_idx):
      name,lab,txt,img,val,arou,e1,e2,e3,e4,e5,e6,e7,e8,e9,intensity,t1,t2,t3,t4,t5,t6 = batch
      name = batch[name]
      lab = batch[lab]
      txt = batch[txt]
      img = batch[img]
      e1 = batch[e1]
      e2 = batch[e2]
      e3 = batch[e3]
      e4 = batch[e4]
      e5 = batch[e5]
      e6 = batch[e6]
      e7 = batch[e7]
      e8 = batch[e8]
      e9 = batch[e9]

      intensity = batch[intensity]

      t1,t2,t3,t4,t5,t6,t7 = torch.unsqueeze(batch[t1],1),torch.unsqueeze(batch[t2],1),\
      torch.unsqueeze(batch[t3],1),torch.unsqueeze(batch[t4],1),torch.unsqueeze(batch[t5],1),\
      torch.unsqueeze(batch[t6],1)
      gt_target = torch.cat((t1,t2,t3,t4,t5,t6,t7),1)
      gt_emotion = torch.cat((torch.unsqueeze(e1,1),torch.unsqueeze(e2,1),torch.unsqueeze(e3,1),torch.unsqueeze(e4,1),torch.unsqueeze(e5,1),torch.unsqueeze(e6,1),\
                              torch.unsqueeze(e7,1),torch.unsqueeze(e8,1),torch.unsqueeze(e9,1)),1)

      logits, a,b,c,d,e,f,g,h,i,j,k,l,m,inten,logit_target,logit_emotion = self.forward(txt,img,rag)
      #self.log('val_acc 1', accuracy_score(lab.detach().cpu().numpy(),np.argmax(logits.detach().cpu().numpy(),axis=-1)))
      for n in name:
        N.append(n)
      append_gt(lab,o_t); append_gt(e1,e1_t); append_gt(e2,e2_t); append_gt(e3,e3_t); append_gt(e4,e4_t); append_gt(e5,e5_t);\
      append_gt(e6,e6_t); append_gt(e7,e7_t); append_gt(e8,e8_t); append_gt(e9,e9_t); append_gt(intensity,i_t);

      append_p(logits,o_p); append_p(a,e1_p); append_p(b,e2_p); append_p(c,e3_p); append_p(d,e4_p); append_p(e,e5_p);\
      append_p(f,e6_p); append_p(g,e7_p); append_p(h,e8_p); append_p(i,e9_p); append_p(inten,i_p);

      tmp = np.argmax(logits.detach().cpu().numpy(),axis=-1)
      loss = self.cross_entropy_loss(logits, lab)
      lab = lab.detach().cpu().numpy()
      self.log('test_acc', accuracy_score(lab,tmp))
      self.log('test f1',f1_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1),average='macro'))
      np.save('multitask_logit_emotion.npy',logit_emotion.detach().cpu().numpy())
      np.save('multitask_logit_offensive.npy',lab)
      np.save('multitask_logit_intensity.npy',inten.detach().cpu().numpy())
      #self.log('test confusion matrix',confusion_matrix(lab,tmp))
      #print(f'confusion matrix {confusion_matrix(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1))}')
      print(f'confusion matrix intensity {confusion_matrix(intensity.detach().cpu().numpy(),np.argmax(inten.detach().cpu().numpy(),axis=-1))}')
      print(f'confusion matrix offensive {confusion_matrix(lab,tmp)}')

      #self.log('test_roc_auc',roc_auc_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1)))
      #self.log('F1',f1_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1)))
      #self.log('recall',recall_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1)))
      #self.log('precision',precision_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1)))
      best_threshold = np.array([0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5])
      y_test = torch.nn.Sigmoid()(logit_emotion)
      y_test = y_test.detach().cpu().numpy()
      y_pred = np.array([[1 if y_test[i][j]>=best_threshold[j] else 0 for j in range(13)] for i in range(len(y_test))])
      #print(y_pred)
      total_correctly_predicted = len([i for i in range(len(y_test)) if (y_test[i]==y_pred[i]).sum() == 13])
      self.log('test_loss', loss)
      #print(total_correctly_predicted)
      pred_e = y_test
      return {'test_loss': loss,
              'test_loss_target': F.binary_cross_entropy_with_logits(logit_target.float(), gt_target.float()),
              'test_loss_emotion_multilabel': F.binary_cross_entropy_with_logits(logit_emotion.float(), gt_emotion.float()),
               'test_acc':f1_score(lab,tmp,average='macro'),
              'test_acc e1': accuracy_score(e1.detach().cpu().numpy(),np.argmax(a.detach().cpu().numpy(),axis=-1)),
              'test_acc e2': accuracy_score(e2.detach().cpu().numpy(),np.argmax(b.detach().cpu().numpy(),axis=-1)),
              'test_acc e3': accuracy_score(e3.detach().cpu().numpy(),np.argmax(c.detach().cpu().numpy(),axis=-1)),
              'test_acc e4': accuracy_score(e4.detach().cpu().numpy(),np.argmax(d.detach().cpu().numpy(),axis=-1)),
              'test_acc e5': accuracy_score(e5.detach().cpu().numpy(),np.argmax(e.detach().cpu().numpy(),axis=-1)),
              'test_acc e6': accuracy_score(e6.detach().cpu().numpy(),np.argmax(f.detach().cpu().numpy(),axis=-1)),
              'test_acc e7': accuracy_score(e7.detach().cpu().numpy(),np.argmax(g.detach().cpu().numpy(),axis=-1)),
              'test_acc e8': accuracy_score(e8.detach().cpu().numpy(),np.argmax(h.detach().cpu().numpy(),axis=-1)),
              'test_acc e9': accuracy_score(e9.detach().cpu().numpy(),np.argmax(i.detach().cpu().numpy(),axis=-1)),
              'test_acc inten': f1_score(intensity.detach().cpu().numpy(),np.argmax(inten.detach().cpu().numpy(),axis=-1),average='macro'),
              'test_acc sarcasm': accuracy_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1)),
              'f1 sarcasm': f1_score(sarcasm.detach().cpu().numpy(),np.argmax(logit_sarcasm.detach().cpu().numpy(),axis=-1),average='macro')}
  def test_epoch_end(self, outputs):
        # OPTIONAL
        outs = []
        outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14 = \
        [],[],[],[],[],[],[],[],[],[],[],[],[],[]
        outs15 = []
        outs16 = []
        outs17 = []
        outs18 = []
        for out in outputs:
          outs15.append(out['test_loss_target'])
          outs.append(out['test_acc'])
          outs1.append(out['test_acc e1'])
          outs2.append(out['test_acc e2'])
          outs3.append(out['test_acc e3'])
          outs4.append(out['test_acc e4'])
          outs5.append(out['test_acc e5'])
          outs6.append(out['test_acc e6'])
          outs7.append(out['test_acc e7'])
          outs8.append(out['test_acc e8'])
          outs9.append(out['test_acc e9'])

          outs14.append(out['test_acc inten'])
          outs16.append(out['test_acc sarcasm'])
          outs17.append(out['test_loss_emotion_multilabel'])


        #print(outs)
        self.log('final test f1', sum(outs)/len(outs))
        """
        self.log('test_acc_all e1', sum(outs1)/len(outs1))
        self.log('test_acc_all e2', sum(outs2)/len(outs2))
        self.log('test_acc_all e3', sum(outs3)/len(outs3))
        self.log('test_acc_all e4', sum(outs4)/len(outs4))
        self.log('test_acc_all e5', sum(outs5)/len(outs5))
        self.log('test_acc_all e6', sum(outs6)/len(outs6))
        self.log('test_acc_all e7', sum(outs7)/len(outs7))
        self.log('test_acc_all e8', sum(outs8)/len(outs8))
        self.log('test_acc_all e9', sum(outs9)/len(outs9))
        self.log('test_acc_all e10', sum(outs10)/len(outs10))
        self.log('test_acc_all e11', sum(outs11)/len(outs11))
        self.log('test_acc_all e12', sum(outs12)/len(outs12))
        self.log('test_acc_all e13', sum(outs13)/len(outs13))
        self.log('test_acc_all inten', sum(outs14)/len(outs14))
        self.log('test_loss_all target', sum(outs15)/len(outs15))
        self.log('test_acc_all sarcasm', sum(outs16)/len(outs16))
        """
        self.log('test_loss_all emo', sum(outs17)/len(outs17))
        #self.log('test_f1_all sarcasm', sum(outs18)/len(outs18))


  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=5e-3)
    return optimizer


class HmDataModule(pl.LightningDataModule):

  def setup(self, stage):



    self.hm_train = t_p
    self.hm_val = v_p
    self.hm_test = te_p


  def train_dataloader(self):
    return DataLoader(self.hm_train, batch_size=64)

  def val_dataloader(self):
    return DataLoader(self.hm_val, batch_size=64)

  def test_dataloader(self):
    return DataLoader(self.hm_test, batch_size=128)

data_module = HmDataModule()


checkpoint_callback = ModelCheckpoint(
     monitor='val_acc_all_offn',
     dirpath='noemo/ckpts/',
     filename='our-ds-ckpt-epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}',
     auto_insert_metric_name=False,
     save_top_k=1,
    mode="max",
 )
all_callbacks = []
all_callbacks.append(checkpoint_callback)
"""
for i in range(1,14):
  tmp_checkpoint_callback = ModelCheckpoint(
      monitor='val_acc_all e{}'.format(i),
      dirpath='noemo/ckpts/e{}'.format(i),
      filename='our-ds-ckpt-best-emo-{}'.format(i),
      auto_insert_metric_name=False,
      save_top_k=1,
      mode="max",
  )
  all_callbacks.append(tmp_checkpoint_callback)
"""
# train
from pytorch_lightning import seed_everything
seed_everything(seed=123, workers=True)
hm_model = Classifier()
gpus = 1 if torch.cuda.is_available() else 0
trainer = pl.Trainer(gpus=gpus,max_epochs=60,callbacks=all_callbacks)
#trainer = pl.Trainer(gpus=gpus,deterministic=True,max_epochs=60,callbacks=all_callbacks)

trainer.fit(hm_model, data_module)


In [ ]:
test_dataloader = DataLoader(dataset=te_p,batch_size=20)
ckpt_path = '/content/LLaVA/mrinal/epoch00-val_f1_all_offn0.00.ckpt' # put ckpt_path according to the path output in the previous cell
trainer.test(dataloaders=test_dataloader,ckpt_path=ckpt_path)